In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import pygame
from typing import List
import time

In [ ]:
env = gym.make("CartPole-v1"
            #    , render_mode="human"
               )

In [ ]:
print(f"this is observation space: {type(env.observation_space)}")
print(f"this is action space: {env.action_space}")

# 0: cart position 1: cart velocity 2: pole angle 3: pole angular velocity
# observation high represents the highest value that the observation can take
print(f"this is observation space high: {env.observation_space.high}")
upper_bound = env.observation_space.high.tolist()

# observation lowest represents the lowest value that the observation can take
print(f"this is observation space low: {env.observation_space.low}")
lower_bound = env.observation_space.low.tolist()

In [ ]:
def random_seed() -> int:
    return random.randint(0, 10000)

seed = random_seed()
print(f"this is the seed: {seed}")
env.action_space.seed(seed)
state = env.reset()[0]
print(state)
print(state[0])
print(env.step(0)[0])

# print(type(env))
# print(env.action_space.n)
# print(env.observation_space.shape)
# print(env.observation_space.shape[0])

### Task 1: need to choose corresponding action according to state of the cartpole

##### Task 1.1: getting discrete state

State space in cart pole is infinite, which means it is continuous

hence we need to discretise the state space if not there will be infinite rows in the state space table

We will need to discretise each state variable into n segments

potentially have many different states being mapped into 1 state, but we can make the states finer inorder to reduce the amount of different states being sent into this table

In [ ]:
# Map continuous state to discrete indices for Q-matrix
def discretiser(
        state: List[float], 
        number_of_bins: List[int], 
        lower_bounds: List[float], 
        upper_bounds:List[float]
        ) -> tuple:
    """
    Discretise the continuous state into discrete indices for the Q-matrix
    
    Args:
    state: List of continuous state variables
    number_of_bins: Number of bins to divide each state variable
    lower_bounds: Lower bounds for each state variable
    upper_bounds: Upper bounds for each state variable
    
    Returns:
    Tuple of indices for each state variable
    """
    cart_position_bin = np.linspace(lower_bounds[0], upper_bounds[0], number_of_bins[0])
    cart_velocity_bin = np.linspace(lower_bounds[1], upper_bounds[1], number_of_bins[1])
    pole_angle_bin = np.linspace(lower_bounds[2], upper_bounds[2], number_of_bins[2])
    pole_angle_velocity_bin = np.linspace(lower_bounds[3], upper_bounds[3], number_of_bins[3])
    
    index_position = np.maximum(np.digitize(state[0], cart_position_bin) - 1, 0)
    index_velocity = np.maximum(np.digitize(state[1], cart_velocity_bin) - 1, 0)
    index_angle = np.maximum(np.digitize(state[2], pole_angle_bin) - 1, 0)
    index_angular_velocity = np.maximum(np.digitize(state[3], pole_angle_velocity_bin) - 1, 0)
    
    return tuple([index_position, index_velocity, index_angle, index_angular_velocity])


##### Task 1.2: initialise Q-learning

In [ ]:
# Initialize the Q-learning parameters and Q-matrix
def initialize_q_learning(
        env, 
        number_of_bins: List[int]
        ) -> tuple:
    """
    Initialize the Q-learning parameters and Q-matrix
    
    Args:
    env: Gym environment
    number_of_bins: Number of bins to divide each state variable
    
    Returns:
    Q-matrix and list to store sum of rewards for each episode
    """
    action_number = env.action_space.n
    # Q-matrix initialized with random values between 0 and 1
    q_matrix = np.random.uniform(low=0, high=1, 
                                size=(number_of_bins[0], number_of_bins[1], 
                                      number_of_bins[2], number_of_bins[3], 
                                      action_number))
    # List to store sum of rewards for each episode
    sum_rewards_episode = []
    
    return q_matrix, sum_rewards_episode

##### Task 1.3: selection of action

In [ ]:
def select_action(
        state:tuple , 
        index: int, 
        q_matrix, 
        action_number: int, 
        epsilon: float, 
        number_of_bins: List[int], 
        lower_bounds: List[float], 
        upper_bounds: List[float]
        ) -> int:
    """
    Select an action using the epsilon-greedy strategy.

    Parameters:
    state (array-like): The current state of the environment.
    index (int): The current episode index.
    q_matrix (ndarray): The Q-value matrix.
    action_number (int): The total number of possible actions.
    epsilon (float): The exploration rate.
    number_of_bins (int): The number of bins for discretizing the state space.
    lower_bounds (array-like): The lower bounds for each dimension of the state space.
    upper_bounds (array-like): The upper bounds for each dimension of the state space.

    Returns:
    int: The selected action.
    """
    # First 500 episodes: select completely random actions for exploration
    if index < 500:
        return np.random.choice(action_number)
    
    # After 7000 episodes: gradually decrease epsilon
    if index > 7000:
        epsilon = 0.999 * epsilon
    
    # Explore (random action) with probability epsilon
    if np.random.random() < epsilon:
        return np.random.choice(action_number)
    
    # Exploit (greedy action) with probability 1-epsilon
    state_index = discretiser(state, number_of_bins, lower_bounds, upper_bounds)
    # Find action(s) with maximum Q-value
    max_actions = np.where(q_matrix[state_index] == np.max(q_matrix[state_index]))[0]
    # Randomly choose one if multiple actions have the same max value
    return np.random.choice(max_actions)

In [ ]:
def simulate_episodes(
        env, 
        alpha: float, 
        gamma: float, 
        epsilon: float, 
        number_episodes: int, 
        number_of_bins: List[int], 
        lower_bounds: List[float], 
        upper_bounds: List[float]
        ) -> tuple:
    """
    Simulates a number of episodes using Q-learning algorithm.

    Parameters:
    env (gym.Env): The environment to simulate.
    alpha (float): The learning rate.
    gamma (float): The discount factor.
    epsilon (float): The exploration rate.
    number_episodes (int): The number of episodes to simulate.
    number_of_bins (int): The number of bins to discretize the state space.
    lower_bounds (list): The lower bounds for each state dimension.
    upper_bounds (list): The upper bounds for each state dimension.

    Returns:
    tuple: A tuple containing the Q-matrix and a list of sum of rewards per episode.
    """
    q_matrix, sum_rewards_episode = initialize_q_learning(env, number_of_bins)
    
    for index_episode in range(number_episodes):
        # List to store rewards for this episode
        rewards_episode = []
        
        # Reset environment at the beginning of each episode
        (state_s, _) = env.reset()
        state_s = list(state_s)
        
        print(f"Simulating episode {index_episode}")
        
        # Run episode until terminal state is reached
        terminal_state = False
        while not terminal_state:
            # Get the discretized index of current state
            state_s_index = discretiser(state_s, number_of_bins, lower_bounds, upper_bounds)
            
            # Select action based on current state
            action_a = select_action(state_s, index_episode, q_matrix, env.action_space.n, 
                                    epsilon, number_of_bins, lower_bounds, upper_bounds)
            
            # Take action and get next state and reward
            (state_s_prime, reward, terminal_state, _, _) = env.step(action_a)
            rewards_episode.append(reward)
            state_s_prime = list(state_s_prime)
            
            # Get index for next state
            state_s_prime_index = discretiser(state_s_prime, number_of_bins, lower_bounds, upper_bounds)
            
            # Get max Q-value for next state
            q_max_prime = np.max(q_matrix[state_s_prime_index])
            
            # Update Q-matrix using Bellman equation
            if not terminal_state:
                error = reward + gamma * q_max_prime - q_matrix[state_s_index + (action_a,)]
            else:
                error = reward - q_matrix[state_s_index + (action_a,)]
                
            q_matrix[state_s_index + (action_a,)] = q_matrix[state_s_index + (action_a,)] + alpha * error
            
            # Set current state to next state
            state_s = state_s_prime
        
        # Print and save episode rewards
        print(f"Sum of rewards {np.sum(rewards_episode)}")
        sum_rewards_episode.append(np.sum(rewards_episode))
    
    return q_matrix, sum_rewards_episode

In [ ]:
# Simulate using the learned optimal policy
def simulate_learned_strategy(
        q_matrix, 
        number_of_bins: List[int], 
        lower_bounds :List[float], 
        upper_bounds: List[float]
        ) -> tuple:
    """
    Simulates the learned optimal policy using the Q-matrix.
    
    Parameters:
    q_matrix (np.ndarray): The Q-matrix learned from Q-learning.
    number_of_bins (list): The number of bins for discretizing the state space.
    lower_bounds (list): The lower bounds for each state dimension.
    upper_bounds (list): The upper bounds for each state dimension.
    
    Returns:
    tuple: A tuple containing the obtained rewards and the environment.
    """
    env1 = gym.make('CartPole-v1', render_mode='human')
    (current_state, _) = env1.reset()
    env1.render()
    time_steps = 1000
    obtained_rewards = []
    
    for time_index in range(time_steps):
        print(time_index)
        # Select greedy action
        state_index = discretiser(current_state, number_of_bins, lower_bounds, upper_bounds)
        max_actions = np.where(q_matrix[state_index] == np.max(q_matrix[state_index]))[0]
        action = np.random.choice(max_actions)
        
        current_state, reward, terminated, _, _ = env1.step(action)
        obtained_rewards.append(reward)
        
        if terminated:
            break
    
    return obtained_rewards, env1

In [ ]:
def train_q_learning(
        env, alpha: float, 
        gamma: float, 
        epsilon: float, 
        number_episodes: int, 
        number_of_bins: List[int], 
        lower_bounds: List[float], 
        upper_bounds: List[float]
        ) -> tuple:
    """
    Train the Q-learning agent.
    
    Parameters:
    env (gym.Env): The environment to train the agent on.
    alpha (float): The learning rate.
    gamma (float): The discount factor.
    epsilon (float): The exploration rate.
    number_episodes (int): The number of episodes to train the agent.
    number_of_bins (list): The number of bins for discretizing the state space.
    lower_bounds (list): The lower bounds for each state dimension.
    upper_bounds (list): The upper bounds for each state dimension.
    
    Returns:
    tuple: A tuple containing the Q-matrix and the rewards history.
    """
    # Train the agent
    print("Starting Q-learning training...")
    q_matrix, rewards_history = simulate_episodes(
        env, alpha, gamma, epsilon, number_episodes, 
        number_of_bins, lower_bounds, upper_bounds
    )
    return q_matrix, rewards_history

In [ ]:
# 1. Create CartPole environment
env = gym.make('CartPole-v1')

# 2. Define Q-learning parameters
alpha = 0.1             # Learning rate
gamma = 0.99            # Discount factor
epsilon = 0.3           # Initial exploration rate
number_episodes = 8000  # Total number of training episodes

# 3. Define discretisation parameters
# Number of bins for discretizing each state dimension
number_of_bins = [10, 10, 10, 10]  

# Define bounds for state variables
lower_bounds = [-2.4, -2, -0.2, -1.5]
upper_bounds = [2.4, 2, 0.2, 1.5]

# 4. Train the agent
q_matrix, rewards_history = train_q_learning(
    env, alpha, gamma, epsilon, number_episodes, 
    number_of_bins, lower_bounds, upper_bounds
)

# 5. Plot learning progress
plt.figure(figsize=(12, 5))
plt.plot(rewards_history)
plt.title('Learning Progress')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.show()


In [ ]:
def test_q_learning(q_matrix, env, number_of_bins, lower_bounds, upper_bounds):
    # Test the trained agent
    state, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        env.render()
        state_discrete = discretiser(state, number_of_bins, lower_bounds, upper_bounds)
        action = np.argmax(q_matrix[state_discrete])
        state, reward, done, _, _ = env.step(action)
        total_reward += reward
    env.close()
    return total_reward

# 6. Test the trained agent
total_reward = test_q_learning(q_matrix, env, number_of_bins, lower_bounds, upper_bounds)
print(f'Total reward during testing: {total_reward}')

### Task 2: run the RL agent 100 times, reset state at the start of each iteration 

### Task 3: Render one episode played by the developed RL agent on Jupyter.